In [27]:
import os
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from opensearchpy.exceptions import TransportError, ConnectionError as OSCxnError

import boto3
from requests_aws4auth import AWS4Auth
from openai import OpenAI, OpenAIError
from config import *
import time
import logging

from email import message_from_string
from email import policy
from email.utils import make_msgid
from src.services.data_extraction import fast_stream_first_n
from src.services.querying import *


Clients

In [41]:
query = "Tell me about Simon!"
ids = []

hits, hit_ids = knn_search(query, retrieved_ids=None)
ids.extend(hit_ids)

for i in hits:
    print(i["_source"]["subject"])
print(f"\nExcluded IDs: {ids}")

hits, hit_ids = knn_search(query, retrieved_ids=ids)
ids.extend(hit_ids)

for i in hits:
    print(i["_source"]["subject"])
print(f"\nExcluded IDs: {ids}")

hits, hit_ids = knn_search(query, retrieved_ids=ids)

ids.extend(hit_ids)

for i in hits:
    print(i["_source"]["subject"])
print(f"\nExcluded IDs: {ids}")

Re: FW: Expected Package from LUK

Excluded IDs: ['t_096f1517e364421c8b2ccadf7edc2ba1@priority-one.com']

Excluded IDs: ['t_096f1517e364421c8b2ccadf7edc2ba1@priority-one.com']

Excluded IDs: ['t_096f1517e364421c8b2ccadf7edc2ba1@priority-one.com']


In [42]:
memory = None
retrieved_ids = None

In [43]:
query = "Who is Sarah Hawkins and what is her role?"

prompt, response, memory, retrieved_ids = answer_query(query, memory, retrieved_ids)

RequestError: RequestError(400, 'search_phase_execution_exception', 'No mapping found for [chunk_index] in order to sort on')

In [31]:
print(response)

NameError: name 'response' is not defined

In [56]:
def reconstruct_thread(index_name, thread_id, max_msgs=1000, os_client=os_client):
    """
    Fetches every full email (with inlined attachments) for a given thread_id,
    sorted by date, and returns a list of header+body strings.
    """
    resp = os_client.search(
        index=index_name,
        body={
            "size": max_msgs,
            "query": {
                "bool": {
                    "must": [
                        {"term": {"thread_id": thread_id}},
                        {"term": {"type": "email"}}
                    ]
                }
            },
            "sort": [{"date": {"order": "asc"}}]
        }
    )

    messages = []
    for hit in resp["hits"]["hits"]:
        src = hit["_source"]
        headers = (
            f"\n\nFrom: {src.get('from')}"
            f"\nTo:   {src.get('to')}"
            f"\nCC:   {src.get('cc')}"
            f"\nDate: {src.get('date')}"
            f"\nSubject: {src.get('subject')}\n\n"
        )
        # `body` now contains original email + all attachment texts
        messages.append(headers + src.get("body", ""))
    return messages


In [57]:
thread_id= "qrw1dshdqpu4.bsyayk95cgmj2@de1wp-sprc-452"

full_text= []
header = "\n\n" + "---- " + "Thread Number " + "XXX" + " ----" + "\nSummary: " + "None" + "\n\n"
text = "".join(reconstruct_thread(INDEX_NAME, thread_id=thread_id))
full_text.append(header + text)

print("".join(full_text))



---- Thread Number XXX ----
Summary: None



From: Natalija Prosic <Natalija.Prosic@denholm-logistics.com>
To:   jason.leadbeatter@redcoat.org.uk
CC:   sarah@redcoat.org.uk
Date: 2025-03-25T11:03:00+00:00
Subject: REDCOAT:  LGP  -  ASHDOD  /  SVFES214094  /  ZIMULGP09081462  /  1X40HC

Hi , Please see attached Booking confirmation for the above ref. Cutt offs: SI: 03/04 15:30 UCR: 03/04 15:30 VGM: 03/04 16:00 Let me know if any issues Kind regards Nat Natalija Prosic Denholm Good Logistics Ltd (Liverpool) 2ND FLOOR, 5 ST PAUL'S SQUARE  OLD HALL STREET  LIVERPOOL L3 9SJ Phone: +441515190770 Email: Natalija.Prosic@denholm-logistics.com Web: www.denholmgoodlogistics.com **PLEASE NOTE** If this e-mail originated from cargowise@goodlogisticsgroup.com it is not a monitored e-mail address so please do not reply. Unless otherwise agreed all business is undertaken subject to the following Standard Trading Conditions (copies available on request)   - For Ship, Port, Liner or Booking agency: In

In [50]:
import os
import json
from config import *


# Target thread_id
target_thread_id = "qrw1dshdqpu4.bsyayk95cgmj2@de1wp-sprc-452"

# Loop through all files in the directory
for filename in os.listdir(email_attachment_dir):
    if filename.endswith(".json"):
        filepath = os.path.join(email_attachment_dir, filename)
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)

            if data.get("thread_id") == target_thread_id:
                print(f"\n--- File: {filename} ---")
                print("thread_id:", data.get("thread_id"))
                print("date     :", data.get("date"))
                print("from     :", data.get("from"))
                print("to       :", data.get("to"))
                print("subject  :", data.get("subject"))
                print("body     :", data.get("body")[:500])
                print("\n\n----------------------------------\n\n")
        except Exception as e:
            print(f"Error reading {filename}: {e}")



--- File: email_02763.json ---
thread_id: qrw1dshdqpu4.bsyayk95cgmj2@de1wp-sprc-452
date     : 2025-04-08T14:52:21+00:00
from     : Jason Leadbeatter <jason.leadbeatter@redcoat.org.uk>
to       : Natalija Prosic <Natalija.Prosic@denholm-logistics.com>
subject  : Re: REDCOAT: LGP - ASHDOD / SVFES214094 / ZIMULGP09081462 / 1X40HC
body     : No problem, thank you.
<END OF MESSAGE>


----------------------------------



--- File: email_02764.json ---
thread_id: qrw1dshdqpu4.bsyayk95cgmj2@de1wp-sprc-452
date     : 2025-04-08T14:46:02+00:00
from     : Natalija Prosic <Natalija.Prosic@denholm-logistics.com>
to       : Jason Leadbeatter <jason.leadbeatter@redcoat.org.uk>
subject  : RE: REDCOAT: LGP - ASHDOD / SVFES214094 / ZIMULGP09081462 / 1X40HC
body     : Hi Jason,    Not yet I've chased ZIM, but they seem to be playing catch-up on documentation.  They're currently behind on issuing the BL.    Best regards  Nat      Natalija Prosic - Export Operations  CargoWise Certified Operator  Phone:

The New no-chunking logic

In [ ]:
import os
import json
from collections import defaultdict
from config import emails_dir, parsed_attachments_dir, email_attachment_dir

def merge_emails_and_attachments(
    emails_dir=emails_dir,
    attachments_dir=parsed_attachments_dir,
    out_dir=email_attachment_dir
):
    """
    For each email JSON in `emails_dir`, find all .txt files in `attachments_dir`
    whose filename embeds that email's message_id (_id_<message_id>_id_*.txt),
    read their text, append under separators, and write a merged JSON to `out_dir`.
    """
    os.makedirs(out_dir, exist_ok=True)

    # Build map: message_id → [parsed txt attachment paths]
    attach_map = defaultdict(list)
    for fn in os.listdir(attachments_dir):
        if not fn.endswith(".txt"):
            continue
        parts = fn.split("_id_")
        if len(parts) < 3:
            continue
        msg_id = parts[1]
        attach_map[msg_id].append(os.path.join(attachments_dir, fn))

    # Process each email
    email_files = [f for f in os.listdir(emails_dir) if f.endswith(".json")]
    for idx, fn in enumerate(email_files, start=1):
        email_path = os.path.join(emails_dir, fn)
        with open(email_path, "r", encoding="utf-8") as f:
            email = json.load(f)

        msg_id = email.get("message_id", "").strip("<>")

        # Start with the original body
        merged_body = email.get("body", "")

        # Append every parsed‐attachment text for this message
        for att_path in attach_map.get(msg_id, []):
            try:
                with open(att_path, "r", encoding="utf-8") as af:
                    att_text = af.read()
                merged_body += (
                    f"\n\n--- Attachment: {os.path.basename(att_path)} ---\n"
                    f"{att_text}"
                )
            except UnicodeDecodeError:
                # skip any malformed text file
                continue

        # Rebuild the document with the merged body
        merged = {
            **{k: v for k, v in email.items() if k != "body"},
            "body": merged_body,
            "doc_id": f"e_{msg_id}"
        }

        out_path = os.path.join(out_dir, fn)
        with open(out_path, "w", encoding="utf-8") as outf:
            json.dump(merged, outf, ensure_ascii=False, indent=2)

        if idx % 100 == 0:
            print(f"   → Merged {idx}/{len(email_files)} emails")

    print(f"Done: merged {len(email_files)} emails → {out_dir}")


In [ ]:
# merge_emails_and_attachments(
#     emails_dir,
#     parsed_attachments_dir,
#     email_attachment_dir
# )

   → Merged 100/5000 emails
   → Merged 200/5000 emails
   → Merged 300/5000 emails
   → Merged 400/5000 emails
   → Merged 500/5000 emails
   → Merged 600/5000 emails
   → Merged 700/5000 emails
   → Merged 800/5000 emails
   → Merged 900/5000 emails
   → Merged 1000/5000 emails
   → Merged 1100/5000 emails
   → Merged 1200/5000 emails
   → Merged 1300/5000 emails
   → Merged 1400/5000 emails
   → Merged 1500/5000 emails
   → Merged 1600/5000 emails
   → Merged 1700/5000 emails
   → Merged 1800/5000 emails
   → Merged 1900/5000 emails
   → Merged 2000/5000 emails
   → Merged 2100/5000 emails
   → Merged 2200/5000 emails
   → Merged 2300/5000 emails
   → Merged 2400/5000 emails
   → Merged 2500/5000 emails
   → Merged 2600/5000 emails
   → Merged 2700/5000 emails
   → Merged 2800/5000 emails
   → Merged 2900/5000 emails
   → Merged 3000/5000 emails
   → Merged 3100/5000 emails
   → Merged 3200/5000 emails
   → Merged 3300/5000 emails
   → Merged 3400/5000 emails
   → Merged 3500/5000 e

In [25]:
email = "email_01356.json"

with open(os.path.join(email_attachment_dir, email), "r", encoding="utf-8") as f:
    print(json.load(f)["body"])

Please find AWB attached    Thank you      *IMPORTANT UPDATE: WE HAVE MOVED ON AUGUST 9th*  *Our new address: Redcoat Express Ltd, Endeavour House, Maxwell Way,  Crawley, West Sussex, RH10 9SY*  Simon Leadbeatter  Operations Manager  simon@redcoat.org.uk  +44 (0)1293 522440  +44 (0)7379 094298    Redcoat Express Ltd, Endeavour House, Maxwell Way, Crawley, West Sussex,  RH10 9SY
<END OF MESSAGE>

--- Attachment: _id_cabgx6kc6afjuleoy81ybdxl+g+ymcgstwea9wfy853hubwbdxw@mail.gmail.com_id_AE240480-iata-original-mawb.txt ---
057
LHR
1025 5906
057 1025 5906
Shipper's Name and Address
Shipper's Account Number
Not Negotiable
Air Waybill
Issued by
AIR FRANCE
45 RUE DE PARIS 95747,
 
FRANCE
TE:
 
INSTRO PRECISION LIMITED
SENTINEL HOUSE, ARTILLERY WAY, DISCOVERY PARK CT13 9FL
 
SANDWICH (GB) UNITED KINGDOM
TE: 01843 268265
Copies 1, 2 and 3 of this Air Waybill are originals and have the same validity.
Consignee's Name and Address
Consignee's Account Number
It is agreed that the goods described her

ASYNC CALLS

In [40]:
import os
import json
import asyncio
from openai import OpenAI
from src.tools.thread_summaries import *

# concurrency limit (tune to your network / rate‐limit)
MAX_CONCURRENT = 20
PROGRESS_STEP = 50

async def summarize_and_write(
    thread_id: str,
    data: dict,
    out_dir: str,
    client: OpenAI,
    sem: asyncio.Semaphore,
):
    async with sem:
        full_text = "\n\n".join(data["texts"])[:15000]
        resp = await client.chat.completions.acreate(
            model=SUMMARY_MODEL,
            messages=[
                {"role": "system",
                 "content": "Write a concise, but detailed 2–3 sentence summary of this email thread."},
                {"role": "user", "content": full_text}
            ],
            temperature=0.2,
        )
        summary = resp.choices[0].message.content.strip()

    first_date   = min(data["dates"]).isoformat()
    last_date    = max(data["dates"]).isoformat()
    subject      = next(iter(data["subjects"]))
    participants = list(data["participants"])
    message_ids  = data["message_ids"]

    thread_doc = {
        "type":         "thread",
        "thread_id":    thread_id,
        "subject":      subject,
        "participants": participants,
        "first_date":   first_date,
        "last_date":    last_date,
        "message_ids":  message_ids,
        "summary_text": summary,
        "doc_id":       f"t_{thread_id}"
    }

    out_path = os.path.join(out_dir, f"{thread_id}.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(thread_doc, f, ensure_ascii=False, indent=2)

async def async_assemble_and_summarize(threads: dict[str, dict], out_dir: str):
    os.makedirs(out_dir, exist_ok=True)
    client = OpenAI(api_key=SECRET_KEY)
    sem    = asyncio.Semaphore(MAX_CONCURRENT)

    # create all tasks
    tasks = [
        asyncio.create_task(summarize_and_write(tid, data, out_dir, client, sem))
        for tid, data in threads.items()
    ]

    total = len(tasks)
    completed = 0

    # as each task completes, print progress
    for coro in asyncio.as_completed(tasks):
        try:
            await coro
        except Exception as e:
            print(f"❌ Error: {e}")
        completed += 1
        if completed % PROGRESS_STEP == 0 or completed == total:
            print(f"   → Summarized {completed}/{total} threads")

def main():
    thread_map   = build_thread_map(emails_dir)
    thread_docs  = build_thread_docs(emails_dir, parsed_attachments_dir, thread_map)

    asyncio.run(
        async_assemble_and_summarize(
            thread_docs,
            thread_documents_dir
        )
    )


main()


RuntimeError: asyncio.run() cannot be called from a running event loop